In [69]:
# Install requests if not already installed
!pip install requests

import requests
import re

# 1. Knowledge Base Setup
cifar_labels = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]
kb_facts = [f"Object({label})" for label in cifar_labels]
kb_facts += ["Animal(cat)", "Animal(dog)", "Animal(bird)", "Animal(deer)", "Animal(frog)", "Animal(horse)"]
kb_facts += ["Vehicle(airplane)", "Vehicle(automobile)", "Vehicle(ship)", "Vehicle(truck)"]

kb_rules = [
    "Living(X) :- Animal(X)",
    "Transportation(X) :- Vehicle(X)"
]
kb = kb_facts + kb_rules

# 2. Symbolic Solver
class BackwardChainer:
    def __init__(self, kb):
        self.facts = [fact for fact in kb if ":-" not in fact]
        self.rules = [rule for rule in kb if ":-" in rule]

    def is_valid_query(self, query):
        return "(" in query and ")" in query and query.count("(") == 1

    def solve(self, query):
        if not self.is_valid_query(query):
            raise ValueError(f"Invalid symbolic format: {query}")
        if query in self.facts:
            return True
        pred, arg = query.split("(")
        arg = arg.rstrip(")")
        for rule in self.rules:
            head, body = rule.split(":-")
            head = head.strip()
            body = body.strip()
            if head.replace("X", arg) == query:
                required_fact = body.replace("X", arg)
                if required_fact in self.facts:
                    return True
        return False

# 3. Together AI LLM Integration
def llm_translate(nl_query):
    api_key = "7fd022143e7b3bb14cd5e1bc4229c354af503e86d8fe6ffbd008a28e88d43cb4"
    url = "https://api.together.xyz/v1/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}"}
    messages = [
        {"role": "system", "content": "Convert English questions to symbolic logic in the form Predicate(entity). Only output the logic statement."},
        {"role": "user", "content": nl_query}
    ]
    data = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": messages,
        "max_tokens": 40,
        "temperature": 0
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()["choices"][0]["message"]["content"].strip()
        match = re.search(r"(Animal|Vehicle)\([a-z]+\)", result)
        return match.group(0) if match else result
    else:
        print("API Error:", response.text)
        return "Object(unknown)"

# 4. Pipeline & Testing
solver = BackwardChainer(kb)

def logiclm_pipeline(nl_query):
    symbolic = llm_translate(nl_query)
    try:
        result = solver.solve(symbolic)
        return result, symbolic
    except:
        return False, symbolic

nl_queries = [
    "Is cat an animal?",
    "Is truck a vehicle?",
    "Is frog an animal?",
    "Is airplane a vehicle?",
    "Is horse an animal?",
    "Is automobile an animal?"
]

print("="*40)
print("Logic-LM Pipeline Results (Together AI)")
print("="*40)
for nl in nl_queries:
    result, symbolic = logiclm_pipeline(nl)
    print(f"Q: {nl}")
    print(f"  Symbolic: {symbolic}")
    print(f"  Answer: {'Yes' if result else 'No'}\n")
print("="*40)


Logic-LM Pipeline Results (Together AI)
Q: Is cat an animal?
  Symbolic: Animal(cat)
  Answer: Yes

Q: Is truck a vehicle?
  Symbolic: Vehicle(truck)
  Answer: Yes

Q: Is frog an animal?
  Symbolic: Animal(frog)
  Answer: Yes

Q: Is airplane a vehicle?
  Symbolic: Vehicle(airplane)
  Answer: Yes

Q: Is horse an animal?
  Symbolic: Animal(horse)
  Answer: Yes

Q: Is automobile an animal?
  Symbolic: ¬Automobile(A)
  Answer: No

